# MOVIES PROJECT

necesary imports

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

get responses from lots of urls <br>
starting with: <br>
https://www.imdb.com/...&start=1 <br>
up to: <br>
https://www.imdb.com/...&start=6651

In [ ]:
responses = []
# for position in range(1, 6652, 50):
for position in range(1, 152, 50):
    request = requests.get(
        f"https://www.imdb.com/search/title/?title_type=feature&num_votes=10000,&countries=us&sort=user_rating,desc&start={position}"
    )
    responses.append(request)
len(responses)

sopify all of them

In [ ]:
soups = []
for response in responses:
    soups.append(BeautifulSoup(response.text, 'html.parser'))
len(soups)


## RELEASE DATES

date cleaning function

In [ ]:
def extract_int(string):
    return re.search(r"\d+", string).group()

traverse the soups in search for dates, create a list of them, serialise it

In [ ]:
movie_release_dates = []
for soup in soups:
    movie_release_dates += soup.findAll('span', class_= 'lister-item-year text-muted unbold')

rel_dates_list = []
for str_date in movie_release_dates:
    rel_dates_list.append( extract_int(str_date.text) )

rel_dates_series = pd.Series(rel_dates_list, name="Release Date")
rel_dates_series

## RUNTIMES

create a time-cleaning function

In [ ]:
def clean_time(time_string):

    time_string = time_string.split()
    time_string = time_string[0]

    return int(time_string)

traverse the soups in search for runtimes, create a list of them, serialise it

In [ ]:
runtimes = []
for soup in soups:
    runtimes += soup.findAll('span', class_="runtime")

movie_runtimes = []
for runtime in runtimes:
    movie_runtimes.append( clean_time(runtime.text) )
runtimes_series = pd.Series(movie_runtimes, name="Runtime")

runtimes_series

## MOVIE NAMES
and addresses for details pages

traverse the soups in search for movie names and urls, create a list of them, serialise it

In [ ]:
movie_headers = []
for soup in soups:
    movie_headers += soup.find_all(name="h3", class_="lister-item-header")

names_list = []
hrefs_list = []     # this is very important for later. they look like this: '/title/tt0111161/'
for header in movie_headers:
    names_list.append(header.contents[3].text)
    hrefs_list.append(header.contents[3]['href'] )

names_series = pd.Series(names_list, name="Movie name")
names_series

## FILMING DATES
this may be tricky

first we get the responses for all the 6600+ movies... 

In [ ]:
href_responses = []
# hl = hrefs_list[:1]
for href in hrefs_list:
    request = requests.get(
        f"https://www.imdb.com{href}locations"
    )
    href_responses.append(request)
len(href_responses)

now let's soupify them...

In [ ]:
href_soups = []
for response in href_responses:
    href_soups.append(BeautifulSoup(response.text, 'html.parser'))
len(href_soups)

and start digging for dates...

In [ ]:
filming_dates = []
for idx, soup in enumerate(href_soups):
    date = soup.find(name="section", id="filming_dates")
    filming_dates.append(date)
len(filming_dates)

In [ ]:
for idx, date in enumerate(filming_dates):
    print(idx, date.contents[3].text.strip())

## MOVIE TYPES

traverse the soups in search for movie names, clean them, create a list of them and serialise it

In [ ]:
genres = []

for soup in soups:
    genres += soup.find_all(name="span", class_="genre")

genres_list = []
for gen in genres:
    string = list(gen)[0].replace("\n", "").strip()
    genres_list.append(string)

genres_series = pd.Series(genres_list, name="Genre")
genres_series

## RATING

traverse the soup, get stuff from it and serialise it in 2 steps

In [ ]:
# <div class="inline-block ratings-imdb-rating" name="ir" data-value="8,3">
ratings = []
for soup in soups:
    ratings += soup.find_all(name="div", attrs={'name':'ir'})

ratings_series = pd.Series([ float(rt.text.strip()) for rt in ratings ], name="Rating")
ratings_series

## PROFIT!

In [ ]:
all_series = [names_series, ratings_series, rel_dates_series, runtimes_series, genres_series]

df = pd.concat(all_series, axis=1)

# %timeit df.loc[df.loc[:,'Genre'].str.contains('Action')]
# %timeit df.loc[df['Genre'].str.contains('Action')]
# %timeit df.loc[ [('Action' in genre) for genre in df.loc[:,'Genre']]]
# %timeit df.loc[ [('Action' in genre) for genre in df['Genre']]]


In [ ]:
df

extract action movies

In [ ]:
action = df.loc[ [('Action' in genre) for genre in df['Genre']]]
action